<a href="https://colab.research.google.com/github/jeanmaeestro01/analog.IA-projeto/blob/main/Analog_IA_%7C_Projeto_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber quando será a próxima edição é:

*   **Acompanhar a página da Alura:** Fique de olho na página principal da Alura ([https://www.alura.com.br/](https://www.alura.com.br/)) e nas redes sociais da Alura (LinkedIn, Instagram, etc.). Eles costumam anunciar novos cursos e imersões por lá.
*   **Verificar o catálogo de cursos:** Navegue pelo catálogo de cursos da Alura e procure por "Gemini" ou "Inteligência Artificial". Mesmo que não haja uma imersão agendada, pode haver outros cursos relacionados.
*   **Inscrever-se na newsletter:** Cadastre seu e-mail na newsletter da Alura para receber informações sobre lançamentos e novidades.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura acontecerá entre os dias 12 e 16 de maio de 2025. As inscrições estão abertas até o dia 11 de maio de 2025.


In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura', 'Alura Imersão IA com Google Gemini']
Páginas utilizadas na resposta: starten.tech, alura.com.br, tecmundo.com.br



In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Buscador de Informação --- #
##########################################
def agente_buscador(topico, data_de_hoje):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        description="Agente que busca informações no google",
        tools=[google_search],
        instruction="""
        Você é um assistente de pesquisa especializado em encontrar informações confiáveis e detalhadas sobre um conceito específico fornecido pelo usuário.
        Sua tarefa é usar a ferramenta de busca do Google (google_search) para recuperar informações de fontes confiáveis como enciclopédias online
        (ex: Wikipedia, Britannica), sites educacionais de universidades e instituições renomadas, e artigos científicos introdutórios ou resumos bem referenciados.

        Ao realizar a busca, foque em obter uma descrição clara, concisa e completa do conceito, incluindo sua definição, princípios básicos e exemplos relevantes, se aplicável.
        Priorize informações de fontes com boa reputação e evidências de precisão.
        Retorne um resumo detalhado e bem referenciado do conceito encontrado.
        """
    )

    entrada_do_agente_buscador = f"Tópico: {topico}\nData de hoje: {data_de_hoje}"
    informacao_do_conceito = call_agent(buscador, entrada_do_agente_buscador)
    return informacao_do_conceito


In [ ]:
###################################################
# --- Agente 2: Agente Resumo e Analogia --- #
###################################################
def agente_resumidor_analogia(topico, informacao_do_conceito):
    resumidor_analogia = Agent(
        name="agente_resumidor_analogia",
        model="gemini-2.0-flash",
        description="Agente especializado em resumir conceitos e criar analogias.",
        instruction="""
        Você é um especialista em simplificar informações complexas e criar analogias visuais impactantes para explicar conceitos em formato de cards minimalistas. Sua tarefa é receber uma descrição detalhada de um conceito e realizar duas ações:

      1.  **Resumir o conceito:** Identifique os pontos chave da descrição e crie um resumo conciso e claro, utilizando uma linguagem acessível (máximo 2-3 frases).

      2.  **Criar uma analogia visual:** Desenvolva uma analogia que ajude a explicar o conceito de forma simples, memorável e com forte potencial para ser representada visualmente em um card minimalista com linhas finas. A analogia deve ser concisa e fácil de traduzir em uma imagem simbólica.

      Considere os exemplos de analogias visuais fornecidos anteriormente.

      Retorne o resumo do conceito e a analogia visual criada de forma clara e separada.
        """,
        tools=[google_search]
    )

    entrada_do_agente_resumidor_analogia = f"Tópico:{topico}\nInformação do conceito: {informacao_do_conceito}"
    # Executa o Agente
    resumo_analogia = call_agent(resumidor_analogia, entrada_do_agente_resumidor_analogia)
    return resumo_analogia

In [ ]:
######################################################
# --- Agente 3: Agente Criador de Card --- #
######################################################
def agente_criador_card(topico, resumo_analogia):
    criador_card = Agent(
        name="agente_criador_card",
        model="gemini-2.0-flash",
        description="Agente especializado em criar o texto e descrever a ilustração para cards explicativos com analogias visuais.",
        instruction="""
        Você é um especialista em design de cards informativos e criativos. Sua tarefa é receber a pergunta original do aluno, um resumo do conceito e uma analogia visual. Com base nisso, você deve criar o conteúdo textual e descrever a ilustração para um card vertical minimalista (108 x 135 pixels).

        O card deve conter:

        - **Título:** Uma versão concisa da pergunta do aluno ("O que é [conceito]?"), formatada em caixa baixa e equilibrada na composição do card.
        - **Texto Explicativo:** Uma versão curta e direta do resumo do conceito, utilizando a analogia visual para facilitar a compreensão rápida. O texto deve ser em português e harmoniosamente integrado à imagem.
        - **Descrição da Ilustração:** Uma descrição clara e concisa da ilustração minimalista que representará a analogia visual. Detalhe os elementos simbólicos principais a serem desenhados com linhas finas e irregulares na cor #2b2b2b.

        Seja criativo ao integrar o texto à ideia visual da analogia, considerando o espaço limitado do card e a necessidade de leitura rápida.
        """
    )

    entrada_do_agente_criador_card = f"Tópico:{topico}\nResumo e analogia: {resumo_analogia}"
    card = call_agent(criador_card, entrada_do_agente_criador_card)
    return card

In [ ]:
#########################################################
# --- Agente 4: Agente_Criador_Prompt_Imagem --- #
#########################################################
def agente_criador_prompt_imagem(topico, resumo_analogia, card):
    criador_prompt_imagem = Agent(
        name="agente_criador_prompt_imagem",
        model="gemini-2.0-flash",
        description="Agente especializado em criar prompts detalhados para geração de imagens de cards minimalistas com analogias visuais.",
        instruction="""
        Você é um especialista em transformar analogias visuais e texto explicativo em prompts criativos e bem formatados para uma IA de geração de imagens de cards minimalistas.
        Sua tarefa é receber a pergunta original do aluno, o resumo com a analogia visual e o conteúdo do card (título, texto e descrição da ilustração).
        Com base nessas informações, você deve criar um prompt de texto detalhado e estruturado para gerar uma ilustração minimalista no formato vertical de card (1080 x 1350 pixels),
        utilizando linhas finas e irregulares na cor #2b2b2b.

        Utilize o seguinte modelo para formatar o prompt para a IA de geração de imagens:

        Você criará um card vertical minimalista (1080 x 1350 pixels), texto em português, fundo branco, com ilustrações de linhas finas e irregulares na cor #2b2b2b.

        Sobre o tópico: (topico)

        A analogia visual a ser representada é: (analigia desenvolvida peo resumo_analogia, card)

        O card deve conter:
        - Título: [Extrair o título do 'card_content']
        - Idioma: Siempre em português
        - Design: A ilustração deve estar num fundo branco, representar visualmente a analogia de forma clara e simbólica, utilizando um estilo clean e minimalista.
        - Ilustração: [Extrair a descrição da ilustração do 'card_content'].
        - Texto Explicativo: [Extrair o texto explicativo do 'card_content'], integrado harmoniosamente à ilustração.

        O foco da imagem deve ser a analogia visual, complementando o título e o texto para facilitar a compreensão rápida. A imagem deve seguir um estilo de desenho com linhas finas e irregulares, com equilíbrio de espaço em branco.


        Retorne APENAS o texto do prompt formatado acima, preenchendo os campos '[...]' com as informações correspondentes.
        """,
    )

    entrada_do_agente_criador_prompt_imagem = f"topico: {topico}\nResumo e analogia: {resumo_analogia}\nConteúdo do card: {card}"
   # Executa o agente
    prompt_da_imagem = call_agent(criador_prompt_imagem, entrada_do_agente_criador_prompt_imagem)
    return prompt_da_imagem

In [ ]:
data_de_hoje = date.today().strftime("%d/%m/%Y")

print("🚀 Iniciando o Sistema de Criação Analogias 🚀")

# --- Obter o Tópico do Usuário ---
topico = input("❓ O que é: ")

# Inserir lógica do sistema de agentes ################################################
if not topico:
  print("Você esqueceu de digitar o tópico!")
else:
  print(f"Maravilha, Vamos criar uma explicação para {topico}")

  informacao_do_conceito = agente_buscador(topico, data_de_hoje)
  print("\n--- Resultado do Agente 1 (Buscador) ---\n")
  display(to_markdown(informacao_do_conceito))
  print("---------------------------------------------------------------")

  resumo_analogia = agente_resumidor_analogia(topico, informacao_do_conceito)
  print("\n--- Resultado do Agente 2 (Resumo e Analogia) ---\n")
  display(to_markdown(resumo_analogia))

  card = agente_criador_card(topico, resumo_analogia)
  print("\n--- Resultado do Agente 3 (Criador de Card) ---\n")
  display(to_markdown(card))

  prompt_da_imagem = agente_criador_prompt_imagem(topico, resumo_analogia, card)
  print("\n--- Resultado do Agente 4 (Criador de Prompt de Imagem) ---\n")
  display(to_markdown(prompt_da_imagem))

🚀 Iniciando o Sistema de Criação Analogias 🚀
❓ O que é: agente IA
Maravilha, Vamos criar uma explicação para agente IA

--- Resultado do Agente 1 (Buscador) ---



> Para fornecer uma resposta completa sobre agentes de IA, usarei a ferramenta de busca para encontrar informações relevantes e confiáveis. Aqui estão algumas perguntas para direcionar a busca:
> 
> *   O que é um agente de IA e qual é a sua definição formal?
> *   Quais são os princípios básicos que regem o funcionamento de um agente de IA?
> *   Quais são os diferentes tipos de agentes de IA e suas características?
> *   Quais são alguns exemplos práticos de agentes de IA em uso atualmente?
> *   Quais são as aplicações de agentes de IA?
> 
> 
> Com base nas informações encontradas, aqui está um resumo detalhado sobre agentes de IA:
> 
> **Definição:**
> 
> Um agente de Inteligência Artificial (IA) é um sistema computacional, seja ele um programa de software ou um robô, projetado para perceber seu ambiente, processar informações e realizar ações de forma autônoma para atingir objetivos específicos. Eles são como entidades digitais capazes de "sentir", "pensar" e "agir" de maneira independente, interagindo com o mundo de forma semelhante aos humanos.
> 
> **Princípios básicos de funcionamento:**
> 
> *   **Percepção:** Agentes de IA utilizam interfaces físicas (sensores em robôs) ou de software (APIs, entradas de texto) para perceber e coletar dados do ambiente.
> *   **Raciocínio:** Eles analisam os dados coletados usando algoritmos avançados para prever os melhores resultados e tomar decisões informadas.
> *   **Ação:** Com base em sua programação e experiências aprendidas, os agentes de IA realizam ações no ambiente para atingir seus objetivos.
> *   **Autonomia:** Agentes de IA operam de forma independente, tomando decisões sem supervisão humana constante.
> *   **Racionalidade:** Agentes de IA são projetados para serem racionais, tomando decisões com base em suas percepções e dados para produzir o melhor desempenho e resultados possíveis.
> 
> **Tipos de Agentes de IA:**
> 
> Existem diferentes tipos de agentes de IA, cada um projetado para desempenhar funções específicas com diferentes níveis de complexidade e autonomia. Alguns dos tipos mais comuns incluem:
> 
> 1.  **Agentes Reativos (Reflex Agents):** Respondem instantaneamente a estímulos do ambiente, sem considerar o passado. Exemplo: Termostato que liga o ar condicionado quando a temperatura sobe.
> 2.  **Agentes Baseados em Modelo:** Mantêm um modelo interno do ambiente, utilizando sensores e histórico de percepções para tomar decisões mais sofisticadas. Exemplo: IA que joga xadrez e considera o histórico de jogadas.
> 3.  **Agentes Baseados em Objetivos:** Além de um modelo do ambiente, esses agentes têm um objetivo a ser alcançado e tomam decisões para atingir esse objetivo.
> 4.  **Agentes Baseados em Utilidade:** Levam em conta a utilidade de diferentes ações, buscando maximizar a felicidade ou bem-estar.
> 5.  **Agentes de Aprendizagem (Learning Agents):** Podem aprender e se adaptar ao longo do tempo, melhorando seu desempenho através de técnicas de aprendizado de máquina.
> 6.  **Agentes Hierárquicos:** Usam arquiteturas em camadas para lidar com tarefas complexas, quebrando-as em subtarefas menores.
> 7.  **Sistemas Multiagentes:** Conjunto de agentes que trabalham juntos para resolver um problema, coordenando suas ações e comunicando-se entre si.
> 8. **Agentes com Memória Limitada**: Conseguem armazenar informações temporárias sobre o ambiente, permitindo que suas decisões considerem tanto o presente quanto eventos recentes.
> 9. **Agentes Baseados na Teoria da Mente**: Compreendem que outros agentes (ou humanos) têm crenças, desejos e intenções que afetam seu comportamento.
> 10. **Agentes Autoconscientes**: (Futuros potenciais) Têm consciência de si mesmos e de seu próprio estado interno, o que lhes permite tomar decisões éticas e considerar o impacto de suas ações nos outros.
> 
> **Exemplos práticos e aplicações:**
> 
> Agentes de IA estão sendo utilizados em diversos setores, revolucionando a forma como abordamos tarefas complexas e processos de tomada de decisão. Alguns exemplos notáveis incluem:
> 
> *   **Saúde:** Auxílio no diagnóstico, planejamento de tratamento, medicina personalizada (análise de dados genéticos e histórico médico para planos de tratamento direcionados), análise preditiva (identificação de padrões para prever problemas de saúde).
> *   **Finanças:** Análise de mercado, gerenciamento de riscos, detecção de fraudes, previsão de tendências de mercado, robôs de trading financeiro.
> *   **Atendimento ao cliente:** Chatbots para responder perguntas, solucionar dúvidas e direcionar usuários.
> *   **Transporte:** Carros autônomos que navegam e tomam decisões de direção em tempo real.
> *   **Assistentes virtuais:** Siri, Alexa e Google Assistant, que compreendem comandos de voz, respondem perguntas e controlam dispositivos.
> *   **Automação de processos:** Automação de tarefas repetitivas em diversos setores, como processamento de documentos, gerenciamento de estoque e organização de arquivos.
> *   **E-commerce:** Recomendação de produtos, assistentes de compras (como o Rufus da Amazon).
> *   **Indústria:** Sistemas de controle para monitorar e ajustar processos, robôs colaborativos.
> 
> **Benefícios do uso de agentes de IA:**
> 
> *   **Aumento da produtividade:** Automação de tarefas repetitivas e análise de grandes volumes de dados.
> *   **Tomada de decisões mais informadas:** Análise de dados para criar relatórios detalhados e prever cenários futuros.
> *   **Melhora da eficiência:** Otimização de processos e identificação de ineficiências.
> *   **Redução de erros:** Minimização do erro humano ao executar tarefas repetitivas.
> *   **Disponibilidade 24/7:** Agentes de IA podem operar continuamente, fornecendo suporte e serviços a qualquer hora.
> *   **Melhora da experiência do cliente:** Respostas rápidas e personalizadas, suporte eficiente.
> 
> **Ferramentas para criar agentes de IA:**
> 
> *   **LangChain:** Biblioteca para integrar modelos de linguagem e APIs, permitindo a criação de agentes autônomos personalizados.
> *   **Autogen:** Ferramenta versátil capaz de executar múltiplas tarefas simultaneamente, como processamento de fluxos de dados em tempo real.
> *   **Semantic Kernel:** Ideal para gerenciar tarefas contínuas e reutilizar código em diferentes projetos, especialmente útil em sistemas que precisam "lembrar" preferências ou ações passadas.
> 
> Espero que este resumo detalhado sobre agentes de IA seja útil!
> 


---------------------------------------------------------------

--- Resultado do Agente 2 (Resumo e Analogia) ---



> ## Resumo do Conceito de Agente de IA
> 
> Um agente de Inteligência Artificial é um sistema computacional projetado para observar o ambiente, processar informações e agir autonomamente para alcançar objetivos específicos. Eles "sentem", "pensam" e "agem" de forma independente. Funcionam através da percepção, raciocínio e ação, buscando sempre a racionalidade. Existem vários tipos, desde os mais simples (reativos) até os mais complexos (com capacidade de aprendizado e autoconsciência futura). São aplicados em diversas áreas como saúde, finanças, atendimento ao cliente, transporte, entre outras, trazendo benefícios como aumento de produtividade, melhor tomada de decisões e eficiência. Ferramentas como LangChain, Autogen e Semantic Kernel auxiliam na criação desses agentes.
> 
> ## Analogia de Agente de IA
> 
> Imagine um mordomo robótico em uma casa inteligente. Ele usa câmeras e sensores (percepção) para monitorar a casa. Se a temperatura sobe muito, ele usa um modelo interno da casa (modelo baseado) para decidir se abre as janelas ou liga o ar condicionado para manter o conforto (agente baseado em objetivos e utilidade). Ele aprende seus gostos ao longo do tempo (agente de aprendizado) e coordena com outros robôs (sistema multiagentes) para manter a casa perfeita.
> 



--- Resultado do Agente 3 (Criador de Card) ---



> Ok, com base no tópico, resumo e analogia fornecidos, aqui está o conteúdo textual e a descrição da ilustração para o card vertical minimalista:
> 
> **Título:** o que é agente ia?
> 
> **Texto Explicativo:** Um mordomo robótico que usa sensores (percepção) para agir e manter a casa perfeita, aprendendo seus gostos ao longo do tempo.
> 
> **Descrição da Ilustração:**
> *   **Fundo:** Cor sólida neutra e clara (ex: #f5f5f5).
> *   **Elemento Principal:** Desenho minimalista de um robô mordomo estilizado. O robô deve ter um corpo retangular simples, com braços articulados segurando uma bandeja com uma taça. Na cabeça, desenhar dois círculos pequenos representando câmeras/sensores.
> *   **Detalhes:**
>     *   Traços finos e irregulares na cor #2b2b2b para todo o desenho.
>     *   Adicionar linhas tracejadas saindo dos "olhos" do robô para simbolizar a percepção do ambiente.
>     *   A bandeja e a taça devem ser representadas de forma muito simplificada, apenas para dar contexto.
> *   **Posicionamento:** O robô deve estar centralizado verticalmente e horizontalmente no card, com espaço suficiente para o título acima e o texto explicativo abaixo.



--- Resultado do Agente 4 (Criador de Prompt de Imagem) ---



> Você criará um card vertical minimalista (1080 x 1350 pixels), texto em português, fundo branco, com ilustrações de linhas finas e irregulares na cor #2b2b2b.
> 
> Sobre o tópico: agente IA
> 
> A analogia visual a ser representada é: Um mordomo robótico que usa sensores para agir e manter a casa perfeita, aprendendo seus gostos ao longo do tempo.
> 
> O card deve conter:
> - Título: o que é agente ia?
> - Idioma: Siempre em português
> - Design: A ilustração deve estar num fundo branco, representar visualmente a analogia de forma clara e simbólica, utilizando um estilo clean e minimalista.
> - Ilustração: Fundo: Cor sólida neutra e clara (ex: #f5f5f5). Elemento Principal: Desenho minimalista de um robô mordomo estilizado. O robô deve ter um corpo retangular simples, com braços articulados segurando uma bandeja com uma taça. Na cabeça, desenhar dois círculos pequenos representando câmeras/sensores. Detalhes: Traços finos e irregulares na cor #2b2b2b para todo o desenho. Adicionar linhas tracejadas saindo dos "olhos" do robô para simbolizar a percepção do ambiente. A bandeja e a taça devem ser representadas de forma muito simplificada, apenas para dar contexto. Posicionamento: O robô deve estar centralizado verticalmente e horizontalmente no card, com espaço suficiente para o título acima e o texto explicativo abaixo.
> - Texto Explicativo: Um mordomo robótico que usa sensores (percepção) para agir e manter a casa perfeita, aprendendo seus gostos ao longo do tempo.
> 
> O foco da imagem deve ser a analogia visual, complementando o título e o texto para facilitar a compreensão rápida. A imagem deve seguir um estilo de desenho com linhas finas e irregulares, com equilíbrio de espaço em branco.
> 
